In [1]:
# IMPORT 

import csv
from biomart import BiomartServer
import pandas as pd
import openpyxl

In [2]:
# DATASET

## biomart_server
server = BiomartServer('http://www.ensembl.org/biomart')
dataset = server.datasets['hsapiens_gene_ensembl']

In [3]:
# FUNCTIONS

def gene_dictionary(index,
                    gene_name,
                    gene_list_number,
                    gene_list_id,
                    source,
                    ensembl_gene_id,
                    ensembl_transcript_id,
                    refseq_mrna,
                    hgnc_symbol,
                    organism,
                    alias,
                    direction_string):
    
    dictionary = {
        'Index' : index,
        'Gene_name' : gene_name,
        'Gene_list_number' : gene_list_number,
        'Gene_list_index' : gene_list_id,
        'Source' : source,
        'Ensembl_gene_id' : ensembl_gene_id,
        'Ensembl_transcript_id' : ensembl_transcript_id,
        'RefSeq_mRNA_id' : refseq_mrna,
        'Organism' : organism,
        'HGNC_symbol' : hgnc_symbol,
        'Alias' : alias,
        'Info' : direction_string
    }
    
    return dictionary


  
def biomartParameters(mgi_symbol):

    attributes = ['ensembl_gene_id',
                  'ensembl_transcript_id',
                  'refseq_mrna']

    filters = {'external_gene_name':[mgi_symbol]}               # gene_name = mgi_symbol
    response = dataset.search({'attributes':attributes,'filters':filters})
    

    # response_convertion
    lines = response.text.split("\n")
    lines = list(filter(lambda l: l != "",lines))
    values = []
    for l in lines:
        temp = []
        raw_values = l.split("\t")
        values.append(raw_values)
        
    return values  



def alias_and_official(ls_notResponse,ls_row_10,ls_row_2):

    ls_response = []
    ls_response2 = []
    
    for notResponse in ls_notResponse:
        word = str(notResponse).lower()

        for i in range(len(ls_row_10)):
            if ls_row_10[i]:
                temp = str(ls_row_10[i]).lower()
                t_strings = temp.split("|")
                if word in t_strings:
                    ls_response.append([word, i, ls_row_1[i]])

    ls_response.insert(0, ['gene_name','alias_index','mgi_id'])         # print(len(ls_response))


    for notResponse in ls_notResponse:
        word = str(notResponse).lower()

        for i in range(len(ls_row_10)):
            if ls_row_10[i]:
                temp = str(ls_row_10[i]).lower()
                if word == temp:
                    ls_response2.append([word, i, ls_row_1[i]])
                    

    ls_response.insert(0, ['gene_name','official_index','mgi_id'])      # print(len(ls_response2))

    ls_response_3 = ls_response
    
    for response in ls_response2:
        if response not in ls_response_3:
            ls_response_3.append(response)
        else: print(response)

    ls_response_3.pop(0)
    ls_response_3.pop(0)
    ls_response.insert(0, ['gene_name','official/alias_index','mgi_id'])
    
    return ls_response_3

def biomartParameters_mgi(gene_name):

    attributes = ['ensembl_gene_id',
                  'ensembl_transcript_id',
                  'refseq_mrna']
    filters = {'uniprot_gn_symbol':[gene_name]}                     # gene_name = mgi_id
    response = dataset.search({'attributes':attributes,'filters':filters})
    
    # response_convertion
    lines = response.text.split("\n")
    lines = list(filter(lambda l: l != "",lines))
    values = []
    for l in lines:
        temp = []
        raw_values = l.split("\t")
        values.append(raw_values)

    return values  


def updateCellswithAlias(mgi_file_path, dictionary_file_path, file_path):
    
    # Load MGI file and open workbook
    wb_mgi = openpyxl.load_workbook(mgi_file_path)
    ws_mgi = wb_mgi.active

    ls_row_alias = [ws_mgi.cell(row=i,column=10).value for i in range(2,ws_mgi.max_row+1)]
    ls_row_g_name = [ws_mgi.cell(row=i,column=2).value for i in range(2,ws_mgi.max_row+1)] 

    # Load dictionary file and open workbook
    workbook = openpyxl.load_workbook(dictionary_file_path)
    sheet = workbook.active

    ls_row_2 = [sheet.cell(row=i,column=2).value for i in range(2,sheet.max_row+1)]
    ls_row_9 = [sheet.cell(row=i,column=11).value for i in range(2,sheet.max_row+1)]

    
    length = len(ls_row_2) + 2

    # Modify the desired cell
    for i in range(length):
        K_cel = 'K' + str(i + 2)

        for k in range(len(ls_row_g_name)):
            try:
                if str(ls_row_g_name[k]).lower() == str(ls_row_2[i]).lower():
                    if ls_row_alias[k] is not None:
                        sheet[K_cel] = ls_row_alias[k]
                    else:
                        continue
            except:
                continue

    # Save the file
    workbook.save(file_path)


In [4]:
# LOAD START FILES

## PUBLICATION SOURCE
wb = openpyxl.load_workbook('../ifpan-GR-database-papers.xlsx')
ws = wb['embryos-scRNA-seq']

publication_geneName = [ws.cell(row=i,column=3).value for i in range(2,ws.max_row+1)]
direction = [ws.cell(row=i,column=6).value for i in range(2,ws.max_row+1)]

## MGI
wb_1 = openpyxl.load_workbook(r'../MGI_EntrezGene.xlsx')
ws_1 = wb_1.active

ls_row_10 = [ws_1.cell(row=i,column=10).value for i in range(2,ws_1.max_row+1)]
ls_row_2 = [ws_1.cell(row=i,column=2).value for i in range(2,ws_1.max_row+1)]
ls_row_1 = [ws_1.cell(row=i,column=1).value for i in range(2,ws_1.max_row+1)]  

In [5]:
# LISTS

dictionary = {}
ls_geneDictionaries = []
ls_notResponse = []
ls_notResponse_after = []


In [6]:
# DICTIONARY

for i in range(len(publication_geneName)):
    gene_name = publication_geneName[i]
    direction_string = direction[i]   # => info
    hgnc_symbol = gene_name
    
    if gene_name != '':

        # VARIABLES
        index = i + 1
        gene_list_number = 29
        gene_list_id =  'all_sig_genes_35948369'      # => cluster
        source = 'PMID: 35948369'
        organism = 'human'
        print(direction_string)
        alias = ''


        ensembl_gene_id_temp = []
        ensembl_transcript_id_temp = []
        refseq_mrna_temp = []

        ls_biomartParameters = biomartParameters(gene_name)
        if not ls_biomartParameters:
            ls_notResponse.append(gene_name)
            print(ls_notResponse)


        for ls in ls_biomartParameters:
            for j in range(3):
                if len(ls) < (j+1):
                    ls.append('')
            ensembl_gene_id_temp.append(ls[0])
            ensembl_transcript_id_temp.append(ls[1])
            refseq_mrna_temp.append(ls[2])  

        ensembl_gene_id = '|'.join(list(set(filter(None, ensembl_gene_id_temp))))
        ensembl_transcript_id = '|'.join(list(set(filter(None, ensembl_transcript_id_temp))))
        refseq_mrna = '|'.join(list(set(filter(None, refseq_mrna_temp))))
        
      
        
        temp_gene_dictionary = gene_dictionary(index,
                                               gene_name,
                                               gene_list_number,
                                               gene_list_id,
                                               source,
                                               ensembl_gene_id,
                                               ensembl_transcript_id, 
                                               refseq_mrna,
                                               hgnc_symbol,
                                               organism,
                                               alias,
                                               direction_string)

        ls_geneDictionaries.append(temp_gene_dictionary)
        print(temp_gene_dictionary)





up
{'Index': 1, 'Gene_name': 'ALPP', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000163283', 'Ensembl_transcript_id': 'ENST00000485563|ENST00000392027|ENST00000474529', 'RefSeq_mRNA_id': 'NM_001632', 'Organism': 'human', 'HGNC_symbol': 'ALPP', 'Alias': '', 'Info': 'up'}
up
{'Index': 2, 'Gene_name': 'ALYREF', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000183684', 'Ensembl_transcript_id': 'ENST00000505490|ENST00000512673|ENST00000511412|ENST00000504015', 'RefSeq_mRNA_id': 'NM_005782', 'Organism': 'human', 'HGNC_symbol': 'ALYREF', 'Alias': '', 'Info': 'up'}
up
{'Index': 3, 'Gene_name': 'ASS1', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000130707', 'Ensembl_transcript_id': 'ENST00000352480|ENST00000372394|ENST00000493984|ENST00000470849|ENST00000467695|EN

{'Index': 21, 'Gene_name': 'DPPA3', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000187569', 'Ensembl_transcript_id': 'ENST00000345088', 'RefSeq_mRNA_id': 'NM_199286', 'Organism': 'human', 'HGNC_symbol': 'DPPA3', 'Alias': '', 'Info': 'up'}
up
{'Index': 22, 'Gene_name': 'EIF3F', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000175390', 'Ensembl_transcript_id': 'ENST00000677179|ENST00000532882|ENST00000678993|ENST00000528763|ENST00000528653|ENST00000530219|ENST00000677795|ENST00000531572|ENST00000651655|ENST00000678132|ENST00000531329|ENST00000677866|ENST00000533626|ENST00000677121|ENST00000640290', 'RefSeq_mRNA_id': 'NM_003754', 'Organism': 'human', 'HGNC_symbol': 'EIF3F', 'Alias': '', 'Info': 'up'}
up
{'Index': 23, 'Gene_name': 'EIF5A', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', '

{'Index': 39, 'Gene_name': 'IL32', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000008517', 'Ensembl_transcript_id': 'ENST00000531965|ENST00000534748|ENST00000008180|ENST00000525377|ENST00000530538|ENST00000548246|ENST00000396887|ENST00000548476|ENST00000526464|ENST00000396890|ENST00000534507|ENST00000382213|ENST00000525643|ENST00000613483|ENST00000525003|ENST00000551513|ENST00000551122|ENST00000530890|ENST00000528163|ENST00000440815|ENST00000529699|ENST00000548652|ENST00000528652|ENST00000552664|ENST00000325568|ENST00000548807|ENST00000549213|ENST00000552356|ENST00000529550|ENST00000444393|ENST00000532086|ENST00000532247|ENST00000533097|ENST00000525228|ENST00000552936', 'RefSeq_mRNA_id': 'NM_001369589|NM_001369588|NM_001308078|NM_001012634|NM_001012631|NM_004221|NM_001012633|NM_001369590|NM_001369591|NM_001369592|NM_001369587|NM_001012635|NM_001012636|NM_001369593|NM_001012632|NM_001376923|NM_001012718|NM_00

['ATP5L2', 'C19orf43', 'CCBL1', 'CCDC109B', 'FAM211A-AS1', 'H1FX', 'H2AFV', 'HIST1H1C', 'HIST1H2AG', 'ILF3-AS1', 'LOC102724023', 'LOC102724159', 'LOC102724592']
{'Index': 54, 'Gene_name': 'LOC102724592', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'human', 'HGNC_symbol': 'LOC102724592', 'Alias': '', 'Info': 'up'}
up
['ATP5L2', 'C19orf43', 'CCBL1', 'CCDC109B', 'FAM211A-AS1', 'H1FX', 'H2AFV', 'HIST1H1C', 'HIST1H2AG', 'ILF3-AS1', 'LOC102724023', 'LOC102724159', 'LOC102724592', 'LOC150776']
{'Index': 55, 'Gene_name': 'LOC150776', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'human', 'HGNC_symbol': 'LOC150776', 'Alias': '', 'Info': 'up'}
up
{'Index': 56, 'Gene_name': 'MBOAT1', 'Gene_list_number': 29, 'Gene_list_index':

{'Index': 71, 'Gene_name': 'NMI', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000123609', 'Ensembl_transcript_id': 'ENST00000243346|ENST00000414946|ENST00000477072|ENST00000491771', 'RefSeq_mRNA_id': 'NM_004688', 'Organism': 'human', 'HGNC_symbol': 'NMI', 'Alias': '', 'Info': 'up'}
up
{'Index': 72, 'Gene_name': 'NOL7', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000225921', 'Ensembl_transcript_id': 'ENST00000474485|ENST00000420088|ENST00000451315', 'RefSeq_mRNA_id': 'NM_016167|NM_001317724', 'Organism': 'human', 'HGNC_symbol': 'NOL7', 'Alias': '', 'Info': 'up'}
up
{'Index': 73, 'Gene_name': 'NSUN4', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000117481', 'Ensembl_transcript_id': 'ENST00000486270|ENST00000307089|ENST00000474844|ENST00000537428|ENST000004

...

{'Index': 2573, 'Gene_name': 'RSBN1', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000081019', 'Ensembl_transcript_id': 'ENST00000261441|ENST00000369581|ENST00000615321|ENST00000476412|ENST00000612242', 'RefSeq_mRNA_id': 'NM_018364', 'Organism': 'human', 'HGNC_symbol': 'RSBN1', 'Alias': '', 'Info': 'up'}
up
{'Index': 2574, 'Gene_name': 'SAFB2', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000130254', 'Ensembl_transcript_id': 'ENST00000585449|ENST00000591310|ENST00000592599|ENST00000587802|ENST00000591123|ENST00000591120|ENST00000590262|ENST00000252542|ENST00000591101|ENST00000590000|ENST00000589925', 'RefSeq_mRNA_id': 'NM_014649', 'Organism': 'human', 'HGNC_symbol': 'SAFB2', 'Alias': '', 'Info': 'up'}
up
{'Index': 2575, 'Gene_name': 'SALL4', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 359483

{'Index': 2589, 'Gene_name': 'SLC7A5', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000103257', 'Ensembl_transcript_id': 'ENST00000565644|ENST00000563489|ENST00000261622', 'RefSeq_mRNA_id': 'NM_003486', 'Organism': 'human', 'HGNC_symbol': 'SLC7A5', 'Alias': '', 'Info': 'up'}
up
{'Index': 2590, 'Gene_name': 'SMARCA4', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000127616', 'Ensembl_transcript_id': 'ENST00000642350|ENST00000645061|ENST00000644737|ENST00000429416|ENST00000589677|ENST00000646593|ENST00000592158|ENST00000646484|ENST00000646183|ENST00000644327|ENST00000586985|ENST00000586122|ENST00000643208|ENST00000646513|ENST00000643857|ENST00000646693|ENST00000444061|ENST00000643549|ENST00000591595|ENST00000586892|ENST00000704345|ENST00000541122|ENST00000344626|ENST00000591545|ENST00000645236|ENST00000644267|ENST00000644290|ENST000005

{'Index': 2603, 'Gene_name': 'ST3GAL1', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000008513', 'Ensembl_transcript_id': 'ENST00000523855|ENST00000522204|ENST00000522285|ENST00000521180|ENST00000518298|ENST00000517668|ENST00000519435|ENST00000523634|ENST00000522873|ENST00000520020|ENST00000648219|ENST00000521627|ENST00000519924|ENST00000523854|ENST00000399640|ENST00000522652', 'RefSeq_mRNA_id': 'NM_173344|NM_003033', 'Organism': 'human', 'HGNC_symbol': 'ST3GAL1', 'Alias': '', 'Info': 'up'}
up
{'Index': 2604, 'Gene_name': 'STAT3', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000168610', 'Ensembl_transcript_id': 'ENST00000389272|ENST00000677421|ENST00000676636|ENST00000471989|ENST00000678792|ENST00000678108|ENST00000678906|ENST00000679014|ENST00000585517|ENST00000678913|ENST00000678043|ENST00000677763|ENST00000677723|ENST00000678535|

{'Index': 2619, 'Gene_name': 'TSKU', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000182704', 'Ensembl_transcript_id': 'ENST00000333090|ENST00000527881|ENST00000533752|ENST00000612930|ENST00000525167', 'RefSeq_mRNA_id': 'NM_001258210|NM_001318477|NM_001318479|NM_015516|NM_001318478', 'Organism': 'human', 'HGNC_symbol': 'TSKU', 'Alias': '', 'Info': 'up'}
up
{'Index': 2620, 'Gene_name': 'TSR1', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000167721', 'Ensembl_transcript_id': 'ENST00000576112|ENST00000575049|ENST00000301364|ENST00000576202|ENST00000571806', 'RefSeq_mRNA_id': 'NM_018128', 'Organism': 'human', 'HGNC_symbol': 'TSR1', 'Alias': '', 'Info': 'up'}
up
{'Index': 2621, 'Gene_name': 'TTF2', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000116830', 'Ensem

{'Index': 2636, 'Gene_name': 'YTHDF3', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000185728', 'Ensembl_transcript_id': 'ENST00000613853|ENST00000621413|ENST00000518373|ENST00000621820|ENST00000615676|ENST00000539294|ENST00000621957|ENST00000519428|ENST00000623280|ENST00000621890|ENST00000522282|ENST00000517371|ENST00000619337|ENST00000612880|ENST00000617200|ENST00000617952', 'RefSeq_mRNA_id': 'NM_001277816|NM_001277818|NM_001277815|NM_001277817|NM_001277813|NM_001277814|NM_152758', 'Organism': 'human', 'HGNC_symbol': 'YTHDF3', 'Alias': '', 'Info': 'up'}
up
{'Index': 2637, 'Gene_name': 'ZBTB43', 'Gene_list_number': 29, 'Gene_list_index': 'all_sig_genes_35948369', 'Source': 'PMID: 35948369', 'Ensembl_gene_id': 'ENSG00000169155', 'Ensembl_transcript_id': 'ENST00000450858|ENST00000373457|ENST00000449886|ENST00000373464|ENST00000497064', 'RefSeq_mRNA_id': 'NM_001135776|NM_014007', 'Organism': 'human', 'HGNC_symb

In [7]:
print(ls_notResponse)

['ATP5L2', 'C19orf43', 'CCBL1', 'CCDC109B', 'FAM211A-AS1', 'H1FX', 'H2AFV', 'HIST1H1C', 'HIST1H2AG', 'ILF3-AS1', 'LOC102724023', 'LOC102724159', 'LOC102724592', 'LOC150776', 'MTRNR2L1', 'RP11-344E13.3', 'RRP7B', 'SNAR-E', 'TRNAA5', 'TRNAR27', 'C11orf49', 'C21orf33', 'CGB', 'FAM49A', 'GCN1L1', 'LINC01021', 'LOC100129034', 'LOC100862671', 'LOC101927284', 'LOC102724257', 'LPHN3', 'MTRNR2L6', 'PVRL1', 'RP11-10A14.5', 'RP11-266L9.2', 'SEPP1', 'SNAR-C3', 'C19orf43', 'C4orf27', 'FAM207A', 'H1FX', 'H2AFY', 'KIAA1161', 'LOC101928115', 'LOC102724023', 'LOC102724548', 'LRRC16A', 'MTRNR2L1', 'PAPD5', 'RP11-344E13.3', 'RP11-538P18.2', 'SNAR-C3', 'SNAR-E', 'TMEM206', 'C21orf33', 'LOC646762', 'LPHN3', 'SELM', 'AC005235.1', 'AC008703.1', 'AC009948.5', 'AC012314.8', 'AC074289.1', 'ACPT', 'AL132709.5', 'BAIAP2-AS1', 'BZRAP1', 'C12orf65', 'C16orf58', 'C17orf62', 'C19orf57', 'C20orf112', 'C20orf194', 'CCDC108', 'CCDC135', 'CXXC11', 'CYHR1', 'DFNB31', 'ENTHD2', 'FAM115C', 'FAM129C', 'FAM134C', 'FAM160A1', 

In [8]:
# SCORES 

### from DICTIONARY to file
data = ls_geneDictionaries
df = pd.DataFrame.from_dict(data)
df.to_excel(r'.\Dictionary.xlsx', index=False)

### from ls_notResponse to file
data1 = ls_notResponse
df1 = pd.DataFrame.from_dict(data1)
df1.to_excel(r'.\notResponse.xlsx', index=False)

# LOAD 
# notRESPONSE
wb_2 = openpyxl.load_workbook(r'.\notResponse.xlsx')
ws_2 = wb_2.active
ls_notResponse_v2 = [ws_2.cell(row=i,column=1).value for i in range(2,ws_2.max_row+1)]


### from alias_and_official to file
data2 = alias_and_official(ls_notResponse, ls_row_10, ls_row_2)
df2 = pd.DataFrame.from_dict(data2)
df2.to_excel(r'.\responsewithMGI.xlsx', index=False)


# LOAD responsewithMGI
wb_3 = openpyxl.load_workbook(r'.\responsewithMGI.xlsx')
ws_3 = wb_3.active
gene_name_v2 = [ws_3.cell(row=i,column=1).value for i in range(3,ws_3.max_row+1)]

['hist1h2ag', 78711, 'MGI:2448293']
['fam115c', 80677, 'MGI:2385258']
['gltscr1', 74168, 'MGI:2154263']
['hist1h4i', 78808, 'MGI:2448432']
['itfg3', 69889, 'MGI:2146854']
['lace1', 74700, 'MGI:2148801']
['pcnxl3', 42194, 'MGI:1861733']
['h2afj', 94797, 'MGI:3606192']
['hist1h2ae', 78709, 'MGI:2448290']
['stra13', 29121, 'MGI:894324']
['tceb3', 39834, 'MGI:1351315']
['aim1', 28469, 'MGI:109544']
['h2afj', 94797, 'MGI:3606192']
['ptpla', 40646, 'MGI:1353592']


In [9]:
print (ls_notResponse)

['ATP5L2', 'C19orf43', 'CCBL1', 'CCDC109B', 'FAM211A-AS1', 'H1FX', 'H2AFV', 'HIST1H1C', 'HIST1H2AG', 'ILF3-AS1', 'LOC102724023', 'LOC102724159', 'LOC102724592', 'LOC150776', 'MTRNR2L1', 'RP11-344E13.3', 'RRP7B', 'SNAR-E', 'TRNAA5', 'TRNAR27', 'C11orf49', 'C21orf33', 'CGB', 'FAM49A', 'GCN1L1', 'LINC01021', 'LOC100129034', 'LOC100862671', 'LOC101927284', 'LOC102724257', 'LPHN3', 'MTRNR2L6', 'PVRL1', 'RP11-10A14.5', 'RP11-266L9.2', 'SEPP1', 'SNAR-C3', 'C19orf43', 'C4orf27', 'FAM207A', 'H1FX', 'H2AFY', 'KIAA1161', 'LOC101928115', 'LOC102724023', 'LOC102724548', 'LRRC16A', 'MTRNR2L1', 'PAPD5', 'RP11-344E13.3', 'RP11-538P18.2', 'SNAR-C3', 'SNAR-E', 'TMEM206', 'C21orf33', 'LOC646762', 'LPHN3', 'SELM', 'AC005235.1', 'AC008703.1', 'AC009948.5', 'AC012314.8', 'AC074289.1', 'ACPT', 'AL132709.5', 'BAIAP2-AS1', 'BZRAP1', 'C12orf65', 'C16orf58', 'C17orf62', 'C19orf57', 'C20orf112', 'C20orf194', 'CCDC108', 'CCDC135', 'CXXC11', 'CYHR1', 'DFNB31', 'ENTHD2', 'FAM115C', 'FAM129C', 'FAM134C', 'FAM160A1', 

In [10]:
print (gene_name_v2)


['atp5l2', 'ccbl1', 'ccdc109b', 'h1fx', 'h2afv', 'hist1h1c', 'hist1h2ag', 'fam49a', 'gcn1l1', 'lphn3', 'pvrl1', 'sepp1', 'h1fx', 'h2afy', 'lrrc16a', 'papd5', 'tmem206', 'lphn3', 'selm', 'acpt', 'bzrap1', 'ccdc108', 'ccdc135', 'cyhr1', 'dfnb31', 'enthd2', 'fam115c', 'fam129c', 'fam134c', 'fam160a1', 'fam198a', 'fam63b', 'gcn1l1', 'gltscr1', 'gpr123', 'gpr126', 'gpr133', 'hist1h4i', 'itfg3', 'lace1', 'lphn1', 'lrrc48', 'mfi2', 'park2', 'pcnxl3', 'vprbp', 'wdr66', 'aes', 'apoa1bp', 'ccdc23', 'ccdc58', 'eftud1', 'ept1', 'fam49b', 'fam60a', 'fam69b', 'h2afj', 'h2afv', 'h2afy', 'hist1h1e', 'hist1h2ae', 'mesdc2', 'mgea5', 'nhp2l1', 'ppap2c', 'ptplad1', 'selt', 'shfm1', 'stra13', 'stra13', 'tceb3', 'tmem206', 'tmem57', 'tsta3', 'usmg5', 'utp11l', 'wbscr22', 'zcchc6', 'aim1', 'aim1', 'bzrap1', 'dirc2', 'gatsl3', 'h1fx', 'h2afj', 'mpp6', 'pvrl3', 'shfm1', 'tmem5', 'fam109a', 'fam73a', 'garem', 'mars', 'mum1', 'prosc', 'ptpla']


In [11]:
# SECOND DICTIONARY

for i in range(len(gene_name_v2)):
    gene_name = gene_name_v2[i]
    print(gene_name)
    hgnc_symbol = gene_name
    
    if gene_name != '':

        # variables
        alias = ''

        ensembl_gene_id_temp = []
        ensembl_transcript_id_temp = []
        refseq_mrna_temp = []
        

        ls_biomartParameters_mgi = biomartParameters_mgi(gene_name)
        
        print(ls_biomartParameters_mgi)
        
        if not ls_biomartParameters_mgi:
            ls_notResponse_after.append(gene_name)


        for ls in ls_biomartParameters_mgi:
            for j in range(3):
                if len(ls) < (j+1):
                    ls.append('')
            ensembl_gene_id_temp.append(ls[0])
            ensembl_transcript_id_temp.append(ls[1])
            refseq_mrna_temp.append(ls[2])  

        ensembl_gene_id = '|'.join(list(set(filter(None, ensembl_gene_id_temp))))
        ensembl_transcript_id = '|'.join(list(set(filter(None, ensembl_transcript_id_temp))))
        refseq_mrna = '|'.join(list(set(filter(None, refseq_mrna_temp))))
        
        
        
        for k in range(len(ls_geneDictionaries)):
            temp = str(ls_geneDictionaries[k]['Gene_name']).lower()

            if temp == gene_name:
                print('response')
                ls_geneDictionaries[k]['Ensembl_gene_id'] = ensembl_gene_id
                ls_geneDictionaries[k]['Ensembl_transcript_id'] = ensembl_transcript_id
                ls_geneDictionaries[k]['RefSeq_mRNA_id'] = refseq_mrna
                ls_geneDictionaries[k]['HGNC_symbol'] = hgnc_symbol
                


atp5l2
[]
response
ccbl1
[['ENSG00000171097', 'ENST00000302586', 'NM_001352989'], ['ENSG00000171097', 'ENST00000302586', 'NM_001352988'], ['ENSG00000171097', 'ENST00000302586', 'NM_001122671'], ['ENSG00000171097', 'ENST00000302586', 'NM_001352991'], ['ENSG00000171097', 'ENST00000302586', 'NM_004059'], ['ENSG00000171097', 'ENST00000302586', 'NM_001352999'], ['ENSG00000171097', 'ENST00000302586', 'NM_001352990'], ['ENSG00000171097', 'ENST00000302586', 'NM_001352992'], ['ENSG00000171097', 'ENST00000302586', 'NM_001352993'], ['ENSG00000171097', 'ENST00000462722', ''], ['ENSG00000171097', 'ENST00000320665', 'NM_001122672'], ['ENSG00000171097', 'ENST00000483599', ''], ['ENSG00000171097', 'ENST00000436267', 'NM_001352997'], ['ENSG00000171097', 'ENST00000436267', 'NM_001352994'], ['ENSG00000171097', 'ENST00000436267', 'NM_001352998'], ['ENSG00000171097', 'ENST00000436267', 'NM_001287390'], ['ENSG00000171097', 'ENST00000466418', ''], ['ENSG00000171097', 'ENST00000451800', ''], ['ENSG00000171097

[]
response
fam69b
[['ENSG00000165716', 'ENST00000371692', 'NM_152421'], ['ENSG00000165716', 'ENST00000371691', '']]
response
h2afj
[['ENSG00000246705', 'ENST00000544848', 'NM_177925'], ['ENSG00000246705', 'ENST00000389078', ''], ['ENSG00000246705', 'ENST00000501744', '']]
response
response
h2afv
[]
response
response
h2afy
[]
response
response
hist1h1e
[['ENSG00000168298', 'ENST00000304218', 'NM_005321']]
response
hist1h2ae
[]
response
mesdc2
[]
response
mgea5
[]
response
nhp2l1
[['ENSG00000100138', 'ENST00000401959', 'NM_001003796'], ['ENSG00000100138', 'ENST00000648161', ''], ['ENSG00000100138', 'ENST00000648674', 'NM_005008'], ['ENSG00000100138', 'ENST00000215956', ''], ['ENSG00000100138', 'ENST00000463675', ''], ['ENSG00000100138', 'ENST00000649479', ''], ['ENSG00000100138', 'ENST00000648350', ''], ['ENSG00000100138', 'ENST00000402458', ''], ['ENSG00000100138', 'ENST00000469028', ''], ['ENSG00000100138', 'ENST00000649722', ''], ['ENSG00000100138', 'ENST00000488571', ''], ['ENSG0000

In [12]:
# SCORES 

### from SECOND DICTIONARY to file

data3 = ls_geneDictionaries
df3 = pd.DataFrame.from_dict(data3)
df3.to_excel(r'.\secondDictionary.xlsx', index=False)
df3.to_csv(r'.\secondDictionary.tsv', sep="\t")

data4 = ls_notResponse_after
df4 = pd.DataFrame.from_dict(data4)
df4.to_excel(r'.\ls_notResponse_after.xlsx', index=False)



In [13]:
# Add ALIAS

mgi_file_path = r'..\MGI_EntrezGene.xlsx'
dictionary_file_path = r'.\secondDictionary.xlsx'
save_file_path = r'.\withAlias.xlsx'


updateCellswithAlias(mgi_file_path, dictionary_file_path, save_file_path)


In [14]:
# Add INFO

def updateCellswithINFO(source_file_path, source_sheet, alias_file_path, info_file_path):
    
    wb_info = openpyxl.load_workbook(source_file_path)
    ws_info = wb_info[source_sheet]
    ls_row_g_name = [ws_info.cell(row=i, column=3).value for i in range(2, ws_info.max_row + 1)]


    wb_alias = openpyxl.load_workbook(alias_file_path)
    ws_alias = wb_alias.active
    ls_row_2 = [ws_alias.cell(row=i, column=2).value for i in range(2, ws_alias.max_row + 1)]


    matched_indices = set()               # to keep track of matched id

    for j, (g_name, row_2) in enumerate(zip(ls_row_g_name, ls_row_2), start=2):
        if j not in matched_indices and str(g_name).lower() == str(row_2).lower():
            print(str(g_name).lower(), str(row_2).lower())
            L_cel = 'L' + str(j)
            tempInfo = str(ws_alias[L_cel].value)
            matched_indices.add(j)        # add matched index to set - avoid duplicate

### the number and type of items in the INFO column depends on the source publication


            ws_alias[L_cel] = ''
            ws_alias[L_cel] = ('Regulation:' + str(tempInfo).upper() + '|'
                            + 'Log2Ratio:' + str(ws_info['E' + str(j)].value) + '|'
                            + 'tissue:' + str(ws_info['H' + str(j)].value) + '|'
                            + 'cell:' + str(ws_info['I' + str(j)].value) + '|'
                            + 'enviroroment:' + str(ws_info['J' + str(j)].value) + '|'
                            + 'treatment:' + str(ws_info['K' + str(j)].value) + '|'
                            + 'dose:' + str(ws_info['L' + str(j)].value) + '|'
                            + 'time:' + str(ws_info['M' + str(j)].value) + '|'
                            + 'FDR_threshold:' + str(ws_info['N' + str(j)].value) + '|'
                            + 'method:' + str(ws_info['O' + str(j)].value) + '|'
                            + 'padj' + str(ws_info['P' + str(j)].value) + '|'
                            + 'statistics:' + str(ws_info['Q' + str(j)].value))

        
        


    # Save the file
    wb_alias.save(info_file_path)



source_file_path = r'..\ifpan-GR-database-papers.xlsx'
source_sheet = 'embryos-scRNA-seq'
alias_file_path = '.\withAlias.xlsx'
info_file_path = r'.\withINFO.xlsx'

updateCellswithINFO(source_file_path, source_sheet, alias_file_path, info_file_path)


alpp alpp
alyref alyref
ass1 ass1
atp5l2 atp5l2
braf braf
c19orf43 c19orf43
c8orf82 c8orf82
cbs cbs
ccbl1 ccbl1
ccdc109b ccdc109b
ccdc137 ccdc137
cdk20 cdk20
cdx2 cdx2
cenpv cenpv
chchd10 chchd10
clic3 clic3
cox7b2 cox7b2
cxcl14 cxcl14
dcaf15 dcaf15
dctpp1 dctpp1
dppa3 dppa3
eif3f eif3f
eif5a eif5a
eif5al1 eif5al1
emc3-as1 emc3-as1
fabp3 fabp3
fam211a-as1 fam211a-as1
fkbp1ap1 fkbp1ap1
fxyd4 fxyd4
gal gal
h1fx h1fx
h2afv h2afv
hes6 hes6
hist1h1c hist1h1c
hist1h2ag hist1h2ag
hmgb1 hmgb1
hnrnpa1 hnrnpa1
hnrnpd hnrnpd
il32 il32
ilf3-as1 ilf3-as1
impdh2 impdh2
ipo7 ipo7
irx2 irx2
isoc2 isoc2
kcnj3 kcnj3
khdrbs3 khdrbs3
kiaa0825 kiaa0825
knop1 knop1
larp7 larp7
leng1 leng1
lmod2 lmod2
loc102724023 loc102724023
loc102724159 loc102724159
loc102724592 loc102724592
loc150776 loc150776
mboat1 mboat1
mir4426 mir4426
mitd1 mitd1
mlst8 mlst8
mphosph8 mphosph8
mrpl23 mrpl23
msh3 msh3
mtch1 mtch1
mtrnr2l1 mtrnr2l1
mzt2a mzt2a
mzt2b mzt2b
ndufc1 ndufc1
nek2 nek2
nfyb nfyb
nkd2 nkd2
nmi nmi
nol7 nol7
ns

In [ ]:
                                ''' + '|'
                                 + '|'
                                +'TTEST.NEURO.ALD:' + str(ws_mgi[S_cel].value) + '|'
                                +'TTEST.NEURO.DHT:' + str(ws_mgi[T_cel].value) + '|'
                                +'TTEST.NEURO.PRG:' + str(ws_mgi[U_cel].value) + '|'
                                +'TTEST.NEURO.ESR:' + str(ws_mgi[V_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.DEX:' + str(ws_mgi[W_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.ALD:' + str(ws_mgi[X_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.DHT:' + str(ws_mgi[Y_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.PRG:' + str(ws_mgi[Z_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.ESR:' + str(ws_mgi[AA_cel].value) + '|'
                                +'FDR_TTEST.NEURO.DEX:' + str(ws_mgi[AB_cel].value) + '|'
                                +'FDR_TTEST.NEURO.ALD:' + str(ws_mgi[AC_cel].value) + '|'
                                +'FDR_TTEST.NEURO.DHT:' + str(ws_mgi[AD_cel].value) + '|'
                                +'FDR_TTEST.NEURO.PRG:' + str(ws_mgi[AE_cel].value) + '|'
                                +'FDR_TTEST.NEURO.ESR:' + str(ws_mgi[AF_cel].value) + '|'
                                +'FOLD.ASTRO.DEX:' + str(ws_mgi[AG_cel].value) + '|'
                                +'FOLD.ASTRO.ALD:' + str(ws_mgi[AH_cel].value) + '|'
                                +'FOLD.ASTRO.DHT:' + str(ws_mgi[AI_cel].value) + '|'
                                +'FOLD.ASTRO.PRG:' + str(ws_mgi[AJ_cel].value) + '|'
                                +'FOLD.ASTRO.ESR:' + str(ws_mgi[AK_cel].value) + '|'
                                +'FOLD.NEURO.DEX:' + str(ws_mgi[AL_cel].value) + '|'
                                +'FOLD.NEURO.ALD:' + str(ws_mgi[AM_cel].value) + '|'
                                +'FOLD.NEURO.DHT:' + str(ws_mgi[AN_cel].value) + '|'
                                +'FOLD.NEURO.PRG:' + str(ws_mgi[AO_cel].value) + '|'
                                +'FOLD.NEURO.ESR:' + str(ws_mgi[AP_cel].value))'''